In [ ]:
!date

# Memory and time

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import matplotlib
import json
import os

matplotlib.rcParams.update({'font.size': 20})

%config InlineBackend.figure_format = 'retina'

In [ ]:
REF = {
    "arabidopsis-SRR8257100_v2": "arabidopsis",
    "fly-SRR8513910_v2": "fly",
    "human_mouse-hgmm1k_v2": "human_mouse",
    "human-SRR8327928_v2": "human",
    "human-SRR8524760_v2": "human",
    "mouse-EMTAB7320_v2": "mouse",
    "mouse-heart1k_v2": "mouse",
    "mouse-SRR6998058_v2": "mouse",
    "mouse-SRR8206317_v2": "mouse",
    "mouse-SRR8599150_v2": "mouse",
    "mouse-SRR8639063_v2": "mouse",
    "rat-SRR7299563_v2": "rat",
    "worm-SRR8611943_v2": "worm",
    "zebrafish-SRR6956073_v2": "zebrafish",
    "human_mouse-hgmm10k_v3": "human_mouse",
    "human_mouse-hgmm1k_v3": "human_mouse",
    "human-pbmc10k_v3": "human",
    "human-pbmc1k_v3": "human",
    "mouse-heart1k_v3": "mouse",
    "mouse-neuron10k_v3": "mouse",
}

In [ ]:
config_all = '../scripts/config_all.json'
with open(config_all) as f:
  addresses = json.load(f)

ref_dir = addresses['ref_dir']
out_dir = addresses['out_dir']

In [ ]:
def get_time(line):
    # returns milliseconds
    t = ":".join(line.split(":")[4:]).strip()
    hours, minutes, seconds = (["0", "0"] + t.split(":"))[-3:]
    hours = int(hours)
    minutes = int(minutes)
    seconds = float(seconds)
    ms = int(3600000 * hours + 60000 * minutes + 1000 * seconds)

    return ms
        
def get_mem(line):
    # returns bytes
    mem = int(line.split(':')[-1].strip())*1000
    return mem

def get_memtime(f):
    with open(f, 'r') as file:
        for idx, line in enumerate(file):
            if idx == 4:
                time = get_time(line)
            if idx == 9:
                mem = get_mem(line)
    return (mem, time)

### load kallisto index time and mem

In [ ]:
# kb files
d = {}

d["index"] = {"time": [], "mem": []}
files = sorted(glob.glob(ref_dir+'/*/kallisto.log'))

mem = []
time = []
for f in files:
    m, t = get_memtime(f)
    mem.append(m)
    time.append(t)
d["index"]['time'] = time
d["index"]['mem'] = mem

In [ ]:
names = [f.split("/")[-2] for f in files]

kb_idx = pd.DataFrame.from_dict({(i,j): d[i][j] 
                           for i in d.keys() 
                           for j in d[i].keys()})
kb_idx.index = names

### load alevin index time and mem

In [ ]:
# al files
d = {}

d["index"] = {"time": [], "mem": []}
files = sorted(glob.glob(ref_dir+'/*/salmon.log'))

mem = []
time = []
for f in files:
    m, t = get_memtime(f)
    mem.append(m)
    time.append(t)
d["index"]['time'] = time
d["index"]['mem'] = mem

In [ ]:
names = [f.split("/")[-2] for f in files]

al_idx = pd.DataFrame.from_dict({(i,j): d[i][j] 
                           for i in d.keys() 
                           for j in d[i].keys()})
al_idx.index = names

### nreads

In [ ]:
run_log = sorted(glob.glob(out_dir+'/kallisto_out/*/run_info.json'))

In [ ]:
nreads = []
for r in run_log:
    with open(r, 'r') as f:
        nreads.append(json.load(f)["n_processed"])

### load kallisto time and mem

In [ ]:
ksteps = ['pseudoalignment', 'sort_output', 'whitelist', 'correct',  'sort_barcodes', 'count']
asteps = ['pseudoalignment', 'permitlist', 'collate', 'quant']

In [ ]:
# kb files
d = {}
for step in ksteps:
    d[step] = {"time": [], "mem": []}
    files = sorted(sorted(glob.glob(out_dir+'kallisto_out/*/'+step+'.log')))
    mem = []
    time = []
    for f in files:
        m, t = get_memtime(f)
        mem.append(m)
        time.append(t)
    d[step]['time'] = time
    d[step]['mem'] = mem

**Convert bustools timing to three main steps: Whitelist - Collate - Quant**

- sort_outout + whitelist -> whitelist (generate-permit-list)
- correct + sort_barcodes -> collate
- count -> quant

In [ ]:
d_kal = {}
d_kal['pseudoalignment'] = d['pseudoalignment']

l_kal = len(d_kal['pseudoalignment']['time'])

mem = []
time = []
for i in range(l_kal):
    t = d['sort_output']['time'][i] + d['whitelist']['time'][i]
    m = max(d['sort_output']['mem'][i], d['whitelist']['mem'][i])
    time.append(t)
    mem.append(m)

d_kal['permitlist'] = {"time": [], "mem": []}
d_kal['permitlist']['time'] = time
d_kal['permitlist']['mem'] = mem

mem = []
time = []
for i in range(l_kal):   
    t = d['correct']['time'][i] + d['sort_barcodes']['time'][i]
    m = max(d['correct']['mem'][i], d['sort_barcodes']['mem'][i])
    time.append(t)
    mem.append(m)
    
d_kal['collate'] = {"time": [], "mem": []}
d_kal['collate']['time'] = time
d_kal['collate']['mem'] = mem

d_kal['quant'] = d['count']

In [ ]:
names = [f.split("/")[-2] for f in files]
kb = pd.DataFrame.from_dict({(i,j): d_kal[i][j] 
                           for i in d_kal.keys() 
                           for j in d_kal[i].keys()})

files = sorted(glob.glob(out_dir+'kallisto_out/*/sc.bus'))
fsize = [os.path.getsize(f) for f in files]

kb.index = names
kb['nreads'] = nreads
kb['fsize', 'mem'] = fsize

In [ ]:
kb['index', 'name'] = kb.index.map(REF)
kb['index', 'time'] = kb['index', 'name'].map(kb_idx['index', 'time'])
kb['index', 'mem'] = kb['index', 'name'].map(kb_idx['index', 'mem'])

### load alevin time and mem

In [ ]:
# alevin files
d = {}
for step in asteps:
    d[step] = {"time": [], "mem": []}
    files = sorted(glob.glob(out_dir+'alevin_out/*/'+step+'.log'))

    mem = []
    time = []
    for f in files:
        m, t = get_memtime(f)
        mem.append(m)
        time.append(t)
    d[step]['time'] = time
    d[step]['mem'] = mem

In [ ]:
names = [f.split("/")[-2] for f in files]
al = pd.DataFrame.from_dict({(i,j): d[i][j] 
                           for i in d.keys() 
                           for j in d[i].keys()})
al.index = names

files = sorted(glob.glob(out_dir+'alevin_out/*/permitlist/map.collated.rad'))
fsize = [os.path.getsize(f) for f in files]

al['nreads'] = nreads
al['fsize', 'mem'] = fsize

In [ ]:
al['index', 'name'] = al.index.map(REF)
al['index', 'time'] = al['index', 'name'].map(al_idx['index', 'time'])
al['index', 'mem']  = al['index', 'name'].map(al_idx['index', 'mem'])

### sum time and take max of mem

In [ ]:
CMDS = {
   'index': {
        "kb": "$ kallisto index",
        'al': "$ salmon index",
        'en': "Purpose: Build mapping index",
    }, 
    'pseudoalignment': {
        "kb": "$ kallisto bus",
        'al': "$ salmon alevin --rad --sketch",
        'en': "Purpose: Perform mapping",
    },  'fsize': {
        "kb": "BUS file",
        'al': "RAD file",
        'en': "Purpose: Store result of mapping",
    },  'permitlist': {
        "kb": "$ bustools sort + bustools whitelist",
        'al': "$ alevin-fry generate-permit-list\n             --knee-distance",
        'en': "Purpose: Error correct of barcodes",
    },  'collate': {
        "kb": "$ bustools correct + bustools sort",
        'al': "$ alevin-fry collate",
        'en': "Purpose: Enable streaming",
    },  'quant': {
        "kb": "$ bustools count",
        'al': "$ alevin-fry quant",
        'en': "Purpose: Generate count matrix",
    },  'tot': {
        "kb": "$ kallisto + bustools pipeline",
        'al': "$ salmon + alevin pipeline",
        'en': "Purpose: Process single cell data",
    }
}

plot_steps = list(CMDS.keys())
plot_steps

time_steps = plot_steps.copy()
mem_steps  = plot_steps.copy()

time_steps.remove('fsize')
time_steps.remove('index')
time_steps.remove('tot')

mem_steps.remove('index')
mem_steps.remove('fsize')
mem_steps.remove('tot')

In [ ]:
plot_steps

In [ ]:
kb['tot', 'time'] = np.sum([kb[step]['time'] for step in time_steps], axis=0)
kb['tot', 'mem']  = kb[[(step, 'mem') for step in mem_steps]].max(axis=1)

al['tot', 'time'] = np.sum([al[step]['time'] for step in time_steps], axis=0)
al['tot', 'mem']  = al[[(step, 'mem') for step in mem_steps]].max(axis=1)

# Plot

In [ ]:
kallisto_color = '#377eb8'
alevin_color = "#e41a1c"
alpha = 0.2

In [ ]:
def plot_time(step, kb, al, ax, label_top=False, label_bottom=True):
    measurement = "time"

    title = f"{step}"

    for nr in kb['nreads']:
        ax.axvline(x=nr,linewidth=1, color='lightgrey', linestyle='--', zorder=1)

    x = kb['nreads']
    y = kb[f'{step}'][f'{measurement}']/1000/60
    amin = min(y)

    ax.scatter(x,y, color=kallisto_color, label="kallisto")

    x = al['nreads']
    y = al[f'{step}'][f'{measurement}']/1000/60
    kmin = min(y)

    ax.scatter(x,y, color=alevin_color, zorder=-1, label= "alevin")

    if label_top:
        ax.set_title(f'{measurement}', fontweight='bold', loc = 'center' )
        ## add labels on top
        ax2 = ax.twiny()
        ax2.set(**{
            "xticks": np.linspace(kb['nreads'].min(), kb['nreads'].max(), kb.shape[0]),
            "xticklabels": kb.sort_values('nreads').index.values,
        })

        for label in ax2.get_xticklabels():
            label.set_rotation(-45)
            label.set_horizontalalignment("right")

        ax2.tick_params(
            axis='x',          # changes apply to the x-axis
            which='minor',      # both major and minor ticks are affected
            bottom=False,      # ticks along the bottom edge are off
            top=False,         # ticks along the top edge are off
            labelbottom=False) # labels along the bottom edge are off

    kwd = {
        'xscale': "log",
        "ylabel": f"Time [min]",
        'xlabel': "",
        'xticklabels': [],
    } 
    if label_bottom:
        kwd.update({
            'xlabel': "Number of reads",
        })
        kwd.pop('xticklabels')

    ax.set(**kwd)

    ax.grid(color='dimgrey', linestyle='-', linewidth=0.5, which="both", alpha = alpha)

    ax.legend(markerscale=2)
    return ax

In [ ]:
def plot_mem(step, kb, al, ax, label_top=False, label_bottom=True):
    measurement = "mem"

    title = f"{step}"

    for nr in kb['nreads']:
        ax.axvline(x=nr,linewidth=1, color='lightgrey', linestyle='--', zorder=1)

    x = kb['nreads']
    y = kb[f'{step}'][f'{measurement}']/10**9
    ax.scatter(x,y, color=kallisto_color, label="kallisto")

    x = al['nreads']
    y = al[f'{step}'][f'{measurement}']/10**9
    ax.scatter(x,y, color=alevin_color, zorder=-1, label= "alevin")


    if label_top:
        ax.set_title(f'{measurement}'+'ory', fontweight='bold', loc = 'center' )
        ## add labels on top
        ax2 = ax.twiny()
        ax2.set(**{
            "xticks": np.linspace(kb['nreads'].min(), kb['nreads'].max(), kb.shape[0]),
            "xticklabels": kb.sort_values('nreads').index.values,
        })

        for label in ax2.get_xticklabels():
            label.set_rotation(-45)
            label.set_horizontalalignment("right")

        ax2.tick_params(
            axis='x',          # changes apply to the x-axis
            which='minor',      # both major and minor ticks are affected
            bottom=False,      # ticks along the bottom edge are off
            top=False,         # ticks along the top edge are off
            labelbottom=False) # labels along the bottom edge are off
    ylabel = "Memory [GB]"
    if step=='fsize': ylabel="Size [GB]"
    kwd = {
        'xscale': "log",
        "ylabel": ylabel,
        "xlabel": "",
        'xticklabels': [],
        'ylim': min(ax.get_ylim()[0], 0)
    }
    if label_bottom:
        kwd.update({
            'xlabel': "Number of reads",
        })
        kwd.pop("xticklabels")
        
    ax.set(**kwd)

    ax.grid(color='dimgrey', linestyle='-', linewidth=0.5, which="both", alpha = alpha)

    ax.legend(markerscale=2)
    return ax

In [ ]:
left, width = .25, .5
bottom, height = .25, .5
right = left + width
top = bottom + height

In [ ]:
colors = [kallisto_color, alevin_color, "black"]

In [ ]:
n = len(plot_steps)
fig = plt.figure(figsize=(15*2, n*3.5))

gs = fig.add_gridspec(n, 3, hspace=0.0)

axs = [
    (fig.add_subplot(gs[i, 0]), fig.add_subplot(gs[i, 1]), fig.add_subplot(gs[i, 2])) for i in range(len(plot_steps))
]

for idx, ((cmd_ax, time_ax, mem_ax), step) in enumerate(zip(axs, plot_steps)):
    label_top=False
    label_bottom=False
    if idx == 0: label_top=True; cmd_ax.set_title('command', fontweight='bold', loc = 'center' );
    if idx == n-1: label_bottom=True;
    
    # add commands to first column    
    cmds = list(CMDS[step].values())
    ncmds = len(cmds)
    for cidx, (c, color) in enumerate(zip(cmds, colors), -1):
        fontfamily = "monospace"
        if cidx==1: fontfamily = "sans"; 
        kwd = {
                "horizontalalignment": "left",
                "verticalalignment": "center",
                "fontfamily": fontfamily,
                "fontweight":  "bold",
                "fontsize": 20,
                "color": color,
                "transform": cmd_ax.transAxes
        }
        x = 0.225
        y = 0.5*(bottom+top)-cidx/(len(cmds)+2)
        cmd_ax.text(x,y,c, **kwd)
    cmd_ax.set_axis_off()
    
    if step == 'fsize':
        plot_mem(step, kb, al, mem_ax, label_top=label_top, label_bottom=label_bottom)
        time_ax.set_axis_off()
    else:
        plot_time(step, kb, al, time_ax, label_top=label_top, label_bottom=label_bottom)
        plot_mem(step, kb, al, mem_ax, label_top=label_top, label_bottom=label_bottom)

fig.savefig(out_dir+'/memtime.pdf', dpi=100, bbox_inches='tight')
fig.show()

In [ ]:
kb['pseudoalignment']['mem']/10**9

# Sum time and max mem

In [ ]:
fig, ax = plt.subplots(figsize=(14*2, 7), ncols=2)

# sum
plot_time('tot', kb, al, ax[0], label_top=True, label_bottom=True)
ax[0].set_title('Sum time', fontweight='bold')
plot_mem('tot', kb, al, ax[1], label_top=True, label_bottom=True)
ax[1].set_title('Max memory', fontweight='bold')

fig.savefig('sum_max_memtime.png', dpi=300, bbox_inches='tight')

fig.show()

In [ ]:
print('kallisto')
kb_time = kb.loc['human-pbmc10k_v3']['tot']['time']/1000/60
kb_index_time = kb.loc['human-pbmc10k_v3']['index']['time']/1000/60
kb_mem = kb.loc['human-pbmc10k_v3']['tot']['mem']/10**9
print(f"{kb_time} min")
print(f"{kb_index_time} min")
print(f"{kb_mem} GB")

In [ ]:
print('alevin')
al_time = al.loc['human-pbmc10k_v3']['tot']['time']/1000/60
al_index_time = al.loc['human-pbmc10k_v3']['index']['time']/1000/60
al_mem = al.loc['human-pbmc10k_v3']['tot']['mem']/10**9

print(f"{al_time} min")
print(f"{al_index_time} min")
print(f"{al_mem} GB")

https://aws.amazon.com/ec2/pricing/on-demand/

In [ ]:
cost_4  = 0.077
cost_8  = 0.154
cost_16 = 0.308
cost_32 = 0.616

In [ ]:
kb_cost = kb_time /60 * cost_8
al_cost = al_time /60 * cost_8

In [ ]:
kb_cost_index = (kb_time+kb_index_time) /60 * cost_16
al_cost_index = (al_time+al_index_time) /60 * cost_8

In [ ]:
kbc = float(f'{kb_cost:,.2f}')
alc = float(f'{al_cost:,.2f}')

print(f'kallisto costs ${kbc}')
print(f'  alevin costs ${alc}')
print(f'kallisto is {kbc/alc:,.0f} times more expensive tha alevin')

In [ ]:
kbc = float(f'{kb_cost_index:,.2f}')
alc = float(f'{al_cost_index:,.2f}')

print(f'kallisto costs ${kbc}')
print(f'  alevin costs ${alc}')
print(f'kallisto is {kbc/alc:,.0f} times more expensive tha alevin when building the index is considered')

In [ ]:
# not in human mouse
max(al[~al.index.str.contains('hgmm')]['tot']['mem'])/10**9

In [ ]:
# human mouse
al.loc['human_mouse-hgmm10k_v3']['tot']['mem']/10**9

In [ ]:
kb.loc['human_mouse-hgmm10k_v3']['tot']['mem']/10**9

In [ ]:
kb_mean_time = kb['tot']['time'].mean()/1000/60
al_mean_time = al['tot']['time'].mean()/1000/60

In [ ]:
print(f"average kb time: {kb_mean_time:,.2f} min")
print(f"average al time: {al_mean_time:,.2f} min")
print(f"kallisto is {kb_mean_time/al_mean_time:,.0f} times slower than alevin")